In [453]:
#init libraries + data
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import math

#data = pd.read_csv('sample_data/mnist_train_small.csv')
data = pd.read_csv('train.csv')

data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 256

In [454]:
#inits weights and biases + activation functions
w_1 = np.random.rand(10, 784)
w_2 = np.random.rand(10, 10)
b_1 = np.random.rand(10)
#w_1 = np.zeros((10,784))
#w_2 = np.zeros((10,10))
#b_1 = np.zeros(10)

def relu(x):
    return (x>0) * x
    
def drelu(x):
    return x > 0

def sigmoid(x):
    return 1/(1 + (math.e)**(-x))

def dsigmoid(x):
    return sigmoid(x) * (1-sigmoid(x))

In [455]:
def feedforward(activation, w, b, a):
  z_out = np.matmul(w,a)
  z_out += b.reshape(b.size)
  n = z_out.size
  a_out = np.copy(z_out)
  if activation == "sigmoid":
    for i in range(n):
      a_out[i] = sigmoid(z_out[i])
  elif activation == "relu":
    for i in range(n):
      a_out[i] = relu(z_out[i])
  elif activation == "none":
    return z_out,a_out
  else:
    return "Unrecognized activation function"
  return z_out,a_out
      
def cost(a, i):
  temp = np.zeros(10)
  temp[int(Y_train[i])] = 1
  return np.sum((a-temp)**2)
  
def backprop(activation,z,a,w,dz):
  dw = np.matmul(dz.reshape((dz.size,1)),a.reshape((1,a.size)))
  #print(dw)
  dz_out = np.matmul(dz.reshape((1,dz.size)),w)
  n = dz_out.size
  if activation == "sigmoid":
    for x in range(n):
      dz_out[0][x] = dz_out[0][x] * dsigmoid(z[x])
  elif activation == "relu":
    for x in range(n):
      dz_out[0][x] = dz_out[0][x] * drelu(z[x])
  else:
    return dz_out.reshape(n),dw
  return dz_out.reshape(n),dw


In [456]:
def relu_epoch(alpha,w_1, w_2, b_1):
  dz_2 = np.ones(10)
  for i in range(4100):
    z_1,a_1 = feedforward("relu",w_1,b_1,X_train[:,i])
    z_2,a_2 = feedforward("relu",w_2,np.zeros(10),a_1) 
    #solution array
    y_2 = np.zeros(10) 
    y_2[int(Y_train[i])] = 1

    da_2 = a_2-y_2
    for x in range(10):
      dz_2[x] = da_2[x] * drelu(z_2[x])
    dz_1, dw_2 = backprop("relu",z_1,a_1,w_2,dz_2)
    dz_0, dw_1 = backprop("none",X_train[:,i],X_train[:,i],w_1,dz_1)
    
    w_1 -= alpha * dw_1
    w_2 -= alpha * dw_2
    b_1 -= alpha * dz_1 
    #print(w_1,w_2,b_1)
  return w_1,w_2,b_1

def sigmoid_epoch(alpha,w_1, w_2, b_1):
  dz_2 = np.ones(10)
  for i in range(2):
    z_1,a_1 = feedforward("sigmoid",w_1,b_1,X_train[:,i])
    z_2,a_2 = feedforward("sigmoid",w_2,np.zeros(10),a_1) 
    #solution array
    y_2 = np.zeros(10) 
    y_2[int(Y_train[i])] = 1
    #print(y_2)
    #dw_2
    da_2 = a_2-y_2
    for x in range(10):
      dz_2[x] = da_2[x] * dsigmoid(z_2[x])
    dz_1, dw_2 = backprop("sigmoid",z_1,a_1,w_2,dz_2)
    dz_0, dw_1 = backprop("none",X_train[:,i],X_train[:,i],w_1,dz_1)
    #print(dw_2!=0,dw_1!=0,dz_1!=0)
    w_1 -= alpha * dw_1
    w_2 -= alpha * dw_2
    b_1 -= alpha * dz_1 
    #print(w_1,w_2,b_1)
  return w_1,w_2,b_1

In [468]:
#print(np.matmul(w_1,X_train[:,1]))
#print(feedforward("relu",w_1,np.zeros(10),X_train[:,1])[1])
#print(feedforward("relu",w_2,np.zeros(10),feedforward("relu",w_1,np.zeros(10),X_train[:,1])[1]))
for i in range(10):
    w_1, w_2, b_1 = relu_epoch(0.0000000001, w_1, w_2, b_1)
    z_1,a_1 = feedforward("relu",w_1,b_1,X_train[:,0])
    z_2,a_2 = feedforward("relu",w_2,np.zeros(10),a_1)
    print(cost(a_2,0))
    print(a_2,Y_train[0])

7.491529273785576
[-0.         -0.         -0.         -0.          0.86204152 -0.
  2.39758497 -0.         -0.         -0.        ] 9
7.4642905306491265
[-0.         -0.         -0.         -0.          0.8583768  -0.
  2.39321537 -0.         -0.         -0.        ] 9
7.437219221607528
[-0.         -0.         -0.         -0.          0.85472652 -0.
  2.38886203 -0.         -0.         -0.        ] 9
7.4103141049665675
[-0.         -0.         -0.         -0.          0.85109058 -0.
  2.38452489 -0.         -0.         -0.        ] 9
7.383569786570878
[-0.         -0.         -0.         -0.          0.84746648 -0.
  2.38020385 -0.         -0.         -0.        ] 9
7.356989129695684
[-0.         -0.         -0.         -0.          0.84385654 -0.
  2.37589883 -0.         -0.         -0.        ] 9
7.330546637645676
[-0.         -0.         -0.         -0.          0.84026067 -0.
  2.37160465 -0.         -0.         -0.        ] 9
7.304251789928342
[-0.         -0.         -0.       